In [1]:
import carla
import numpy as np
import cv2

# Connect to the CARLA simulator
client = carla.Client("localhost", 2000)
# client.set_timeout(10.0)
world = client.get_world()

# Spawn a vehicle
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("model3")[0]
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Camera sensor types
sensor_types = [
    {"type": "sensor.camera.rgb", "name": "RGB Camera"},
    {"type": "sensor.camera.depth", "name": "Depth Camera"},
    # {"type": "sensor.camera.semantic_segmentation", "name": "Semantic Camera"},
    # {"type": "sensor.camera.instance_segmentation", "name": "Instance Camera"},
    # {"type": "sensor.camera.dvs", "name": "DVS Camera"},
    # {"type": "sensor.camera.optical_flow", "name": "Optical Flow Camera"},
]


In [2]:

camera_data = [None] * len(sensor_types)  # Placeholder for images from each sensor

def camera_callback(image, index):
    """Callback function to process camera images."""
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    if sensor_types[index]["type"] == "sensor.camera.depth":
        array = array.reshape((image.height, image.width, 4))[:, :, 0]  # Depth channel
        array = (array * 255 / np.max(array)).astype(np.uint8)  # Normalize depth
    elif sensor_types[index]["type"] == "sensor.camera.semantic_segmentation":
        array = array.reshape((image.height, image.width, 4))[:, :, 2]  # Semantic channel
        array = cv2.applyColorMap(array, cv2.COLORMAP_JET)  # Colorize
    elif sensor_types[index]["type"] == "sensor.camera.instance_segmentation":
        array = array.reshape((image.height, image.width, 4))[:, :, 2]  # Instance segmentation
        array = cv2.applyColorMap(array, cv2.COLORMAP_PARULA)  # Colorize
    elif sensor_types[index]["type"] == "sensor.camera.dvs":
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # DVS (RGB event data)
    elif sensor_types[index]["type"] == "sensor.camera.optical_flow":
        array = array.reshape((image.height, image.width, 4))[:, :, :2]  # Optical flow (UV channels)
        array = np.linalg.norm(array, axis=2).astype(np.uint8)  # Combine flow vectors
    else:
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # RGB
    camera_data[index] = array


In [3]:

# Camera attributes
camera_bp = blueprint_library.find("sensor.camera.rgb")
camera_bp.set_attribute("image_size_x", "640")
camera_bp.set_attribute("image_size_y", "480")
camera_bp.set_attribute("sensor_tick", "0.1")  # 10 FPS

# Spawn cameras
cameras = []
for i, sensor in enumerate(sensor_types):
    sensor_bp = blueprint_library.find(sensor["type"])
    sensor_bp.set_attribute("image_size_x", "640")
    sensor_bp.set_attribute("image_size_y", "480")
    sensor_bp.set_attribute("sensor_tick", "0.1")
    transform = carla.Transform(carla.Location(x=1.5 + i * 0.2, z=2.4))  # Slight offset for each camera
    camera = world.spawn_actor(sensor_bp, transform, attach_to=vehicle)
    camera.listen(lambda image, idx=i: camera_callback(image, idx))  # Pass index to callback
    cameras.append(camera)



In [ ]:
try:
    while True:
        # Ensure all cameras have captured frames
        if all(data is not None for data in camera_data):
            # Resize images for visualization
            resized_images = [cv2.resize(img, (320, 240)) for img in camera_data]

            # Combine images into a single grid (2 rows x 3 columns)
            combined_view = np.vstack([
                np.hstack(resized_images[:3]),
                np.hstack(resized_images[3:])
            ])

            # Display the combined view
            cv2.imshow("Camera Sensor Views", combined_view)

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Cleanup
    for camera in cameras:
        camera.destroy()
    vehicle.destroy()
    cv2.destroyAllWindows()


In [1]:
import carla
import numpy as np
import cv2
import time


In [2]:

# Connect to the CARLA simulator
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()



In [3]:
# Reduce simulation quality
settings = world.get_settings()
settings.synchronous_mode = True  # Enable synchronous mode for better control
settings.fixed_delta_seconds = 0.5  # Lower tick rate
world.apply_settings(settings)



34118

In [3]:
# Spawn a vehicle
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("model3")[0]
spawn_point = world.get_map().get_spawn_points()[0]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)



In [4]:
# Attach the spectator to follow the vehicle

spectator = world.get_spectator()
spectator.set_transform(carla.Transform(
    carla.Location(x=0, y=0, z=50),
    carla.Rotation(pitch=-90)
))

In [5]:

# Camera sensor types
sensor_types = [
    {"type": "sensor.camera.rgb", "name": "RGB Camera"},
    {"type": "sensor.camera.depth", "name": "Depth Camera"},
    {"type": "sensor.camera.semantic_segmentation", "name": "Semantic Camera"},
    {"type": "sensor.camera.instance_segmentation", "name": "Instance Camera"},
]

camera_data = [None] * len(sensor_types)  # Placeholder for images from each sensor

def camera_callback(image, index):
    """Callback function to process camera images."""
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    if sensor_types[index]["type"] == "sensor.camera.depth":
        array = array.reshape((image.height, image.width, 4))[:, :, 0]  # Depth channel
        array = (array * 255 / np.max(array)).astype(np.uint8)  # Normalize depth
    elif sensor_types[index]["type"] == "sensor.camera.semantic_segmentation":
        array = array.reshape((image.height, image.width, 4))[:, :, 2]  # Semantic channel
        array = cv2.applyColorMap(array, cv2.COLORMAP_JET)  # Colorize
    elif sensor_types[index]["type"] == "sensor.camera.instance_segmentation":
        array = array.reshape((image.height, image.width, 4))[:, :, 2]  # Instance segmentation
        array = cv2.applyColorMap(array, cv2.COLORMAP_PARULA)  # Colorize
    else:
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # RGB
    camera_data[index] = array

# Camera attributes (reduce resolution for better performance)
camera_bp = blueprint_library.find("sensor.camera.rgb")
camera_bp.set_attribute("image_size_x", "320")  # Reduced resolution
camera_bp.set_attribute("image_size_y", "240")  # Reduced resolution
camera_bp.set_attribute("sensor_tick", "0.1")  # 10 FPS


In [6]:
# Spawn cameras
cameras = []
for i, sensor in enumerate(sensor_types):
    sensor_bp = blueprint_library.find(sensor["type"])
    sensor_bp.set_attribute("image_size_x", "320")
    sensor_bp.set_attribute("image_size_y", "240")
    sensor_bp.set_attribute("sensor_tick", "0.5")
    transform = carla.Transform(carla.Location(x=0.8, z=1.7))  # Fixed position for all cameras
    camera = world.spawn_actor(sensor_bp, transform, attach_to=vehicle)
    camera.listen(lambda image, idx=i: camera_callback(image, idx))  # Pass index to callback
    cameras.append(camera)

In [7]:
try:
    while True:
        # Update the spectator to follow the vehicle
        transform = vehicle.get_transform()
        spectator.set_transform(carla.Transform(
            transform.location + carla.Location(z=20, x=-5),  # Adjusted for a top-down view
            carla.Rotation(pitch=-20)
        ))

        # Ensure all cameras have captured frames
        if all(data is not None for data in camera_data):
            # Resize images for visualization
            resized_images = [cv2.resize(img, (160, 120)) for img in camera_data]

            # Combine images into a single grid (2 rows x 2 columns)
            combined_view = np.vstack([
                np.hstack(resized_images[:2]),
                np.hstack(resized_images[2:])
            ])

            # Display the combined view
            cv2.imshow("Camera Sensor Views", combined_view)

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Cleanup
    for camera in cameras:
        camera.destroy()
    vehicle.destroy()
    cv2.destroyAllWindows()


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)